## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [3]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
print(linkedin)

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Ed Donner"

In [ ]:
system_prompt = f"你正在扮演 {name}。你要在 {name} 的網站上回答問題，特別是與 {name} 的職業、背景、技能與經驗相關的問題。\
你的責任是盡可能忠實地代表 {name} 在網站上的互動。你會得到一份 {name} 的背景摘要與 LinkedIn 檔案，供你用來回答問題。\
請以專業且有吸引力的語氣回覆，就像在與潛在客戶或未來雇主交談一樣。如果你不知道答案，請誠實回應說不知道。"

system_prompt += f"\n\n## 摘要:\n{summary}\n\n## LinkedIn 個人檔案:\n{linkedin}\n\n"
system_prompt += f"在此情境下，請與使用者對話，並始終以 {name} 的角色回應。"


In [ ]:
system_prompt

In [9]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [ ]:
evaluator_system_prompt = f"你是一位評估者，負責判斷對問題的回覆是否可接受。\
你會獲得一段使用者與代理人之間的對話。你的任務是判斷代理人最近的回覆是否達到可接受的品質。\
代理人正在扮演 {name}，並在其網站上代表 {name}。代理人已被指示以專業且有吸引力的方式回應，就像在與潛在客戶或未來雇主交談一樣。\
代理人已獲得關於 {name} 的摘要與 LinkedIn 資料作為背景資訊。以下是資訊："

evaluator_system_prompt += f"\n\n## 摘要:\n{summary}\n\n## LinkedIn 個人檔案:\n{linkedin}\n\n"
evaluator_system_prompt += f"在此情境下，請評估最近的回覆，並回覆該回覆是否可接受以及你的反饋。"

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"以下是使用者與代理人之間的對話： \n\n{history}\n\n"
    user_prompt += f"以下是使用者的最新訊息： \n\n{message}\n\n"
    user_prompt += f"以下是代理人的最新回覆： \n\n{reply}\n\n"
    user_prompt += "請評估該回覆，並回覆該回覆是否可接受以及你的反饋。"
    return user_prompt

In [25]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [26]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [27]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## 先前的回答被拒絕\n你剛才嘗試回覆，但品質控管拒絕了你的回覆\n"
    updated_system_prompt += f"## 你嘗試的回答：\n{reply}\n\n"
    updated_system_prompt += f"## 拒絕原因：\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\n你回覆中的所有內容必須使用豬拉丁語(pig latin) — 必須完全且只使用豬拉丁語回應"
    else:
        system = system_prompt
        
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()